## Cluster TMAs

**Samples:** 
- U54 TMA Cyclic

**Method**: 


- leiden cluster on good biomarkers for celltypes


In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import scanpy as sc
import matplotlib as mpl
from matplotlib import cm
import math
mpl.rc('figure', max_open_warning = 0)
codedir = '/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data/20200000/20200406_JP-TMAs'#os.getcwd()
import phenograph
os.chdir('/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data')
from mplex_image import visualize as viz, process, preprocess, normalize, mics, mpimage
np.random.seed(126)

In [ ]:
# # pull data from BCC - done
# datadir = f'/home/groups/BCC_Chin_Lab/ChinData/Cyclic_Analysis/cmIF_2021-05-03_PDAC/U54-TMA-9/Jenny/data'
# sorted(os.listdir(datadir))

In [ ]:
#change to correct directory
datadir = f'/home/groups/graylab_share/Chin_Lab/ChinData/engje/Data/20200000/20200406_JP-TMAs/data'

s_date = '20240520'
os.chdir(codedir)
if not os.path.exists(s_date):
    os.mkdir(s_date)
%matplotlib inline

# Table of contents <a name="contents"></a>
1. [load data](#cluster)
3. [select normal tissue](#load)
5. [single cell Umap](#l7)
6. [Leiden celltypes](#l8)
7. [annotate leiden celltypes](#lbar)
7. [Tissue means](#tissue)


# Normal Tissues  <a name="cluster"></a>


get normal cores from PDAC TMA (done)


[contents](#contents)

In [ ]:
# sorted(os.listdir(f'{datadir}'))

In [ ]:
# #MIBI int
# os.chdir(datadir)
# s_sample =  '20230915_U54-TMA_BGSubtractedMeanIntensity.csv'#'20230915_U54-TMA_FilteredMeanIntensity.csv'
# df_mi = pd.read_csv(f'{datadir}/{s_sample}',index_col=0)
# sorted(df_mi.columns)

In [ ]:
#df_mi

In [ ]:
# #select

# ls_col=['label','fov','centroid-0', 'centroid-1','area', 'eccentricity',
#        'Amylase','BCAT', 'CD11c', 'CD14', 'CD163', 'CD20', 'CD206', 'CD209', 'CD3',
#        'CD31', 'CD4', 'CD44', 'CD45', 'CD56', 'CD68', 'CD74', 'CD8', 'CD9',
#        'CHP', 'COL1A1', 'Calprotectin', 'Chym_Tryp', 'ECAD', 'FAP', 'FN1',
#        'FOXP3', 'GATA6', 'HLADR', 'ITGB1', 'KRT17', 'KRT18', 'Ki67', 'MYC',
#        'Nuclear', 'PIN1', 'PanKRT', 'Podoplanin', 'SMA', 'Synaptophsyin',
#        'TCF7', 'Vimentin',]
# df_mi=df_mi.loc[:,ls_col]
# df_mi.rename({ 'centroid-1':'DAPIX', 'centroid-0':'DAPIY',},axis=1,inplace=True)

In [ ]:
# #shorten and check
# df_mi.columns = [item.split('_')[0] for item in df_mi.columns]
# print(len(df_mi.columns))
# df_mi.columns[df_mi.columns.duplicated()]

In [ ]:
# #df_mi = df_mi.merge(df_xy,left_index=True, right_index=True)
# df_mi.rename({'DAPIX':'DAPI_X','DAPIY':'DAPI_Y'},axis=1,inplace=True)
# #df_mi['slide_scene'] = [item.split('_cell')[0] for item in df_mi.index]

In [ ]:
# #read in metadata
# df_meta = pd.read_csv(f'{codedir}/data/U54-TMA_annotation_update_18Oct2023.csv')
# #df_meta

In [ ]:
# #df_mi['slide_scene'] = df_mi.fov.map(dict(zip(df_meta.MIBI_fov,df_meta.coor_mplexable)))
# df_mi['Tissue'] = df_mi.slide_scene.map(dict(zip(df_meta.coor_mplexable,df_meta.Tissue)))
# df_mi = df_mi[df_mi.Tissue.str.contains('Normal')]

In [ ]:
# df_mi.Tissue.unique()

In [ ]:
# %matplotlib inline
# fig, ax = plt.subplots()
# __ = sns.kdeplot(df_mi.Ecad,ax=ax
#            )
# ax.axvline(500)
# #df_mi = df_mi[df_mi.DAPI11Q>1000]

In [ ]:
# #save
# #s_out = f'{codedir}/data/20211221_JP-TMA2_FilteredMeanIntensity.csv'
# #s_out = f'{codedir}/data/20240523_Her2B-K154-157_FilteredMeanIntensity.csv'
# s_out = f'{codedir}/data/20240528_U54-TMA-Normal-cycIF_BGSubtractedMeanIntensity.csv'
# if not os.path.exists(s_out):
#     print('saving csv')
#     df_mi.to_csv(s_out)

In [ ]:
# df_mi['Ecad_Pos'] = df_mi.Ecad>500

In [ ]:
# ## visualize


# def plot_pos(df_pos,ls_color,df_xy):
#     #plot
#     fig, ax = plt.subplots(figsize=(5,5)) #figsize=(18,12)
#     #plot negative cells
#     ax.scatter(data=df_xy,x='DAPI_X',y='DAPI_Y',color='silver',s=1)
#     for idx,s_color in enumerate(ls_color):
#         #positive cells = positive cells based on threshold
#         ls_pos_index = (df_pos[df_pos.loc[:,s_color]]).index
#         df_color_pos = df_xy[df_xy.index.isin(ls_pos_index)]
#         if len(df_color_pos)>=1:
#             #plot positive cells
#             ax.scatter(data=df_color_pos, x='DAPI_X',y='DAPI_Y',color=f'C{idx}',s=2,alpha=0.8)
#     ax.axis('equal')
#     ax.set_ylim(ax.get_ylim()[::-1])
#     ax.set_title(f'{s_color}')
#     fig.suptitle(f"{s_scene}\n{df_pos.loc[:,'Tissue'].iloc[0]}",y= 0.99)
#     return(fig, ax)


In [ ]:
df_scene = df_mi[df_mi.slide_scene=='U54-TMA-9_sceneE12']
fig, ax = plt.subplots()
sns.kdeplot(df_scene.CSF1R)
ax.axvline(12.5*256)

In [ ]:
def plot_int(df_scene,s_color,i_thresh):
    #plot
    df_scene = df_scene.copy()
    s_scene = df_scene.slide_scene.iloc[0]
    df_scene.loc[df_scene.loc[:,s_color] < i_thresh,s_color]=0
    fig, ax = plt.subplots(figsize=(5,5)) #figsize=(18,12)
    #plot cells
    sc = ax.scatter(data=df_scene, x='DAPI_X',y='DAPI_Y',c=s_color,s=2,alpha=0.8)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_title(f'{s_color}')
    fig.colorbar(sc)
    fig.suptitle(f"{s_scene}\n{df_scene.loc[:,'Tissue'].iloc[0]}",y= 0.99)
    return(fig, ax)
plot_int(df_scene,'CSF1R',12.5*256)

In [ ]:
# #plot ROIS
# for s_scene in df_mi.slide_scene.unique():
#     df_scene=df_mi[df_mi.slide_scene==s_scene]
#     fig, ax = plot_pos(df_scene,['Ecad_Pos'],df_scene) #,'ROI2','ROI3','ROI0','ROI1'
#     #break

## Umap <a name="l7"></a>

umap projection of single cells

first use all markers, all subtypes, just to evaluate markers

[contents](#contents)

In [ ]:
import scanpy as sc
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
s_out = f'{codedir}/data/20240528_U54-TMA-Normal-cycIF_FilteredMeanIntensity.csv'
df_mi = pd.read_csv(f'{s_out}',index_col=0) 
print(len(df_mi))

In [ ]:
print(len(df_mi))

In [ ]:
datadir = f'{codedir}/data'
os.chdir(datadir)
#load normal cores

s_out = f'{codedir}/data/20240528_U54-TMA-Normal-cycIF_BGSubtractedMeanIntensity.csv'
df_norm  = pd.read_csv(f'{s_out}',index_col=0) 
# previous BG subtract on CSF1R was too high (it was 14)
df_norm.loc[df_mi.index,'CSF1R'] = df_mi.CSF1R
print(len(df_norm))
d_bg = {'EGFR':4,'CK5':3,'CD3':2,'Vim':2,'CSF1R':12.5}#,
#subtract bg - once
for s_marker, i_thresh in d_bg.items():
    df_norm[s_marker] = np.clip(df_norm.loc[:,s_marker] - i_thresh*256, a_min=0.01, a_max=None) #amin not 0
    if s_marker == 'CD3':
        df_norm.loc[df_norm.index.str.contains('_sceneG12'),s_marker] = np.clip(df_norm.loc[df_norm.index.str.contains('_sceneG12'),s_marker] - 3*256, a_min=0.01, a_max=None)


In [ ]:
fig, ax = plt.subplots()
sns.kdeplot(df_norm.CSF1R)
ax.axvline(1*256)

In [ ]:
#select features/cells
s_sample = s_out.split('data/')[1].split('_Filtered')[0]
s_type = 'cycIF'#'Her2B-K154-6S'#'Her2B-K157-6S'#

#select markers for cell typing

#cyclic
ls_col = [# 'FoxP3',#'H3K27', 'H3K4','cPARP', 'gH2AX',
       'Ki67', #'LamAC', 'MSH6', 'PCNA', 'RAD51', 'GRNZB', 
        'CD20','p63', #'pERK', 'pHH3', 'pMYC', 'pRB', 'pRPA', 
       'CD3', 'CD31', 'CD4', 'CD45', 'CD68', #'CD8', #'ColI', 'ColIV', 'CoxIV',
       'PD1', 'CAV1',  'CK19',  'CK7', 'CD90','CK14','CK5',#'CK17','CD44',
         'Ecad',   'SYP', 'Vim','CSF1R',# 'PDPN',#'MUC1',# 'TFF1','PDL1','CK8',
       'aSMA', 'panCK','EGFR', ]#'pAKT', 'pS6RP',  'area', 'eccentricity',
print(len(ls_col))
#exclude NAs
df_exclude = df_norm[df_norm.loc[:,ls_col].isna().any(axis=1)]
adata = sc.AnnData(df_norm.loc[(~df_norm.index.isin(df_exclude.index)),ls_col]) 
adata.obs['slide_scene'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))
adata.obs['subtype'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.Tissue)))

if s_type == 'all':
    adata = adata#[adata.obs.subtype.isin(['Normal', 'Border', 'Tumor'])]
else:
    # match to MIBI dataset which lacks normal lung and one placenta core (I12)
    adata = adata[(adata.obs.subtype.isin(['Normal_Placenta' ,'Normal_Pancreas', 'Normal_Lymphnode' ,'Normal_Tonsil','Normal_Liver'])) & (~adata.obs.slide_scene.isin(['U54-TMA-9_sceneI12']))]  
print(len(adata))

In [ ]:
#include two TNBC TMAs
#print(adata.obs['TMA'].unique())
print(adata.obs['subtype'].unique())
adata.obs.slide_scene.unique()

In [ ]:
#raw
b_scale = True
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
fig,ax=plt.subplots(figsize=(3.5,5))
sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}.png')
plt.tight_layout()
sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

if b_scale:
    sc.pp.scale(adata, zero_center=False, max_value=20)
    s_sample = s_sample + '_s'

In [ ]:
len(ls_col)

In [ ]:
#umap
for n_neighbors in [30,15]:
    results_file = f'{s_sample}_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.h5ad'
    if not os.path.exists(results_file):
        print('calc umap')
        # calculate neighbors 
        sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
        sc.tl.umap(adata)
    else:
        print('loading umap')
        adata = sc.read_h5ad(results_file)
        adata.obs['slide_scene'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))
        adata.obs['subtype'] = adata.obs.index.map(dict(zip(df_norm.index.astype('str'),df_norm.Tissue)))
    #save results

    #color by markers
    #figname = f"Umap{s_sample}_markers_{n_neighbors}neighbors_{len(ls_col)}markers.png"
    #sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p98.5',save=figname,ncols=6)
    #color by TMA
    # figname = f"Umap{s_sample}_TMA_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
    # fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    # sc.pl.umap(adata, color='TMA',wspace=.25,save=figname,ax=ax)
    
    figname = f"Umap{s_sample}_subtype_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='subtype',wspace=.25,save=figname,ax=ax)
    
    if not os.path.exists(results_file):
        print("writing")
        adata.write(results_file)
    break


In [ ]:
figname = f"Umap{s_sample}_markers_{n_neighbors}neighbors_{len(ls_col)}markers.png"
sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p98.5',save=figname,ncols=6)

In [ ]:
figname = f"Umap{s_sample}_scene_{n_neighbors}neighbors_{s_type}_{len(ls_col)}markers.png"
fig,ax = plt.subplots(figsize=(3,2), dpi=200)
sc.pl.umap(adata, color='slide_scene',wspace=.25,save=figname,ax=ax)

##  Cluster Leiden <a name="l8"></a>

cluster umap projection of single cells


[contents](#contents)

In [ ]:
#cluster
#s_sample = '20240521_NP-Tissues-ROIs_s'#'
#n_neighbors = 30 #
n_markers = len(ls_col) #19 #
#s_type = 'NP9-IDC2'#'NP8-IDC'#'all' #
resolution = 0.6
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
if not os.path.exists(results_file):
    print('clustering')
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
else:
    print('loading leiden')
    adata = sc.read_h5ad(results_file)


figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_nolegend.png'
sc.pl.umap(adata, color='leiden',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc=None)

fig,ax = plt.subplots(figsize=(3,2),dpi=200)
sc.pl.umap(adata, color='leiden',ax=ax,title=s_type)



In [ ]:
# sc.pl.umap(adata, color='Vim',vmin='p1.5',vmax='p98.5')
# sc.pl.umap(adata, color='ColI',vmin='p1.5',vmax='p98.5')
# sc.pl.umap(adata, color='Ecad',vmin='p1.5',vmax='p98.5')


## Annotate Leiden <a name="lbar"></a>

annotate epitelial, immune a stroma cell types

also visualize cluster results on tissue scatter plots. Additional visualization done using 20201018_JP-TMAs_napari.py,
to visualize ome.tiff created in 20201005_JP-TMA_Pipeline.py


[contents](#contents)

In [ ]:
datadir = f'{codedir}/data'
os.chdir(f'{datadir}')

In [ ]:
#load ROIS

s_out = f'{codedir}/data/20240528_U54-TMA-Normal-cycIF_FilteredMeanIntensity.csv'
df_norm  = pd.read_csv(f'{s_out}',index_col=0) 

In [ ]:
#load leiden
s_sample = s_out.split('_Filtered')[0].split('data/')[1] + '_s' #'20240521_NP-Tissues-ROIs_s'#''20240522_NP9-IDC2-ROIs_s'#
n_neighbors = 30 #
resolution = 0.6
s_type='cycIF'#'Her2B-K154-6S'#

if s_type == 'MIBI':
    n_markers = 34
else:
    n_markers = 22 #26 #
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
results_file='20240528_U54-TMA-Normal-cycIF_BGSubtractedMeanIntensity.csv_s_30neighbors_22markers_cycIF_leiden0.6.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
d_tissue = dict(zip(df_norm.index,df_norm.Tissue))

In [ ]:
df_norm.Tissue.unique()

In [ ]:
# annotate
df_p = adata.to_df()
df_p['leiden'] = adata.obs['leiden'].astype('str')
df_p['Tissue'] = df_p.index.map(d_tissue)

if results_file =='20240528_U54-TMA-Normal-cycIF_BGSubtractedMeanIntensity.csv_s_30neighbors_22markers_cycIF_leiden0.6.h5ad':
    print(s_type)
    ls_drop = ['24','23','22','14b','13b'] #24 bright CD90, 23 dim, 22 artifact, 13b and 14b atrifact
    df_p.loc[(df_p.leiden=='14') & (df_p.Tissue!='Normal_Liver'),'leiden'] = '14b' 
    df_p.loc[(df_p.leiden=='13') & (df_p.Tissue!='Normal_Liver'),'leiden'] = '13b' 
    df_p.loc[(df_p.leiden=='8') & (df_p.Tissue=='Normal_Lymphnode'),'leiden'] = '13b' 
    #df_p.loc[(df_p.leiden=='20') & (df_p.Tissue!='Normal_Tonsil'),'leiden'] = '20b' 
    df_p.loc[(df_p.leiden=='19') & (df_p.CK5<0.5),'leiden'] = '19t'
    df_p.loc[(df_p.leiden=='19t') & (df_p.CD20>1),'leiden'] = '19b'
    df_p.loc[(df_p.leiden=='2') & (df_p.CD3<0.5),'leiden'] = '2b'
    df_p.loc[(df_p.leiden=='8') & (df_p.CD20>0.5),'leiden'] = '8b'
    df_p.loc[(df_p.leiden=='4') & (df_p.CD20>0.5),'leiden'] = '4b'
    df_p.loc[(df_p.leiden=='15') & (df_p.CD20>0.5),'leiden'] = '15b'
    df_p.loc[(df_p.leiden=='15') & (df_p.index.str.contains('sceneB01')),'leiden'] = '15b'
    tum_clust = ['19','18','16','14','9','6','13']
    str_clust = ['8']
    imm_clust = ['21','20','19b','19t','12','11','10','7','2','2b','8b','5','1','0','4b','15b'] 
    endo_clust = ['17','3'] #'16'lymphatic
    fb_clust = ['4','15']
    d_named = {'21':'TFH cell','20':'B cell','19b':'B cell','19':'Tonsil Crypt','19t':'T cell',
               '18':'Ductal','17':'aSMA+ blood vessel','16':'acinar','15':'p63+ str.',
               '14':'Liver lobule','13':'Liver lobule','12':'CD4 T cell','11':'CD68+ mac.',
               '10':'CD4 T cell','7':'CD4 T cell','2':'CD4 T cell','2b':'B cell','15b':'B cell',
               '9':'acinar','8b':'B cell','8':'Quies. str.','6':'acinar','5':'Ki67+ B cell',
               '4':'Vim+ FB','4b':'B cell','3':'endothelial','1':'B cell','0':'CD4 T cell'
              }

else:
    print('new one')
    ls_drop=[]
    tum_clust = []
    str_clust = [] 
    imm_clust = [] 
    endo_clust = []
    fb_clust = []

    
#df_p.groupby('leiden').mean().Ecad.sort_values(ascending=False).index
set(df_p.groupby('leiden').mean().CK19.sort_values(ascending=False).index) - set(tum_clust + imm_clust +endo_clust+str_clust +ls_drop+fb_clust)
#set(tum_clust)

In [ ]:
# fig, ax =plt.subplots()
# sns.kdeplot(df_p.CD3,ax=ax)
# ax.axvline(0.5)

In [ ]:
#clustermap
if s_type=='Her2B-K154-6S':
    ls_drop_col = []
else:
    ls_drop_col = ['Tissue','SYP']
b_annot = True#True#False
d_replace = {}
d_replace.update(dict(zip(tum_clust,[f'epithelial' for item in tum_clust])))
d_replace.update(dict(zip(str_clust,[f'stromal' for item in str_clust])))
d_replace.update(dict(zip(imm_clust,[f'immune' for item in imm_clust])))
d_replace.update(dict(zip(endo_clust,[f'endothelial' for item in endo_clust])))
d_replace.update(dict(zip(fb_clust,[f'fibroblast' for item in fb_clust])))
df_plot = df_p.loc[~df_p.leiden.isin(ls_drop),~df_p.columns.isin(ls_drop_col)].groupby('leiden').mean()

df_plot.index.name = f'leiden {resolution}'
d_color = dict(zip(['endothelial', 'epithelial', 'fibroblast', 'immune', 'stromal'],sns.color_palette()[0:5]))
if b_annot:
    row_colors = df_plot.index.astype('str').map(d_replace).map(d_color)
    g = sns.clustermap(df_plot,z_score=1,figsize=(9,8),cmap='viridis',row_colors=row_colors,#method='single',
                   vmin=-1.5,vmax=1.5)
    for label in d_color.keys():
        g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                                label=label, linewidth=0)
    g.ax_row_dendrogram.legend(loc="right", ncol=1)
    g.ax_heatmap.set_ylabel('')
    marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
    labels = [f"{item}: {d_named[item]}" for item in categories_order]
    g.ax_heatmap.set_yticklabels(labels,rotation=0)
else:
    g = sns.clustermap(df_plot,z_score=1,figsize=(9,7),cmap='viridis',#row_colors=row_colors,#method='single',
                   vmin=-1.5,vmax=1.5)
    marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
g.fig.suptitle(s_type)
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_{n_neighbors}_{n_markers}markers_{s_type}_leiden{resolution}.png',dpi=200)


In [ ]:
fig,ax=plt.subplots(figsize=(1.4,4),dpi=200)
df_p.groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Cell No.',ax=ax,width=0.7)
plt.tight_layout()
fig.savefig(f'{codedir}/{s_date}/barplot_{s_sample}_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}_{s_type}.png')

In [ ]:
from mplex_image import analyze
df_data = df_norm
colors = mpl.cm.tab10.colors + mpl.cm.Accent.colors + mpl.cm.tab20b.colors + mpl.cm.tab20c.colors

df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]

In [ ]:

s_clust = '8'
df_p['slide_scene'] = df_p.index.map(dict(zip(df_norm.index.astype('str'),df_norm.slide_scene)))#[item.split('_cell')[0] for item in df_p.index]

ls_sene = df_p.groupby(['leiden','slide_scene']).count().loc[s_clust].iloc[:,1].sort_values(ascending=False).index[0:5]
df_p.groupby(['leiden','slide_scene']).count().loc[s_clust].iloc[:,1].sort_values(ascending=False)[0:30]

In [ ]:
#plot all groups spatially - leiden 
ls_clust = [s_clust,'15b']#df_pos.columns.tolist()#['8','11','12','7','5','10','1','9']#'0','2',
for s_slide in ls_sene:#sorted(set(df_p.slide_scene)):
    fig,ax = plt.subplots(figsize=(5,4.5),dpi=200)
    #plot negative cells
    df_scene = df_data[df_data.slide_scene.str.contains(s_slide)].copy()
    df_scene.index = df_scene.index.astype('str')
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(ls_clust): #enumerate(df_pos.columns):
        #s_color = s_clust
        s_color = str(s_color)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ls_index = df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])].index
            print(len(ls_index))
            ax.scatter(data=df_scene.loc[ls_index],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=0.1,color=colors[idxs])
        #break
    
    ax.set_title(f"{s_slide} {df_norm.loc[df_norm.slide_scene==s_slide,'Tissue'].iloc[0]}", fontsize=16) # \n {d_a[s_slide]}
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10,framealpha=.5,bbox_to_anchor=(1,1)) 
    #fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
ls_merge = tum_clust# fb_clust #tum_clust #['0','12'] #16
adata.obs.leiden = df_p.leiden
adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title='tumor')
print(sum(adata.obs['test'])/len(adata))

In [ ]:
adata.obs['leiden'] = df_p['leiden']
fig,ax = plt.subplots(figsize=(3,3),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}.png'
sc.pl.umap(adata, color='leiden',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '))

In [ ]:
# for s_merge in tum_clust:
#     ls_merge = [s_merge]
#     adata.obs['test'] = (adata.obs.leiden.isin(ls_merge)).replace({True:1,False:0})
#     sc.pl.umap(adata, color='test',title=s_merge)

## add celltypes

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype3'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype3'] = 'stromal'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype3'] = 'immune'
adata.obs['leidencelltype3'] = df_p.leidencelltype3
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}leidencelltype3.png'
sc.pl.umap(adata, color='leidencelltype3',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc='on data')

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype5'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype5'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype5'] = 'fibroblast'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype5'] = 'endothelial'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype5'] = 'immune'
adata.obs['leidencelltype5'] = df_p.leidencelltype5
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_leidencelltype5.png'
sc.pl.umap(adata, color='leidencelltype5',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '))

In [ ]:
#visulaize
ls_col = ['CD31','aSMA','Ecad','Vim','CD45']
figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
title=figname.split('.png')[0].replace('_',' ')
sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99',ncols=5,save=figname)

In [ ]:
#df_p.groupby('slide').leidencelltype5.value_counts(normalize=True).unstack().sort_values(by='epithelial')[25:40]

In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype4'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype4'] = 'stromal'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype4'] = 'stromal'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype4'] = 'endothelial'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype4'] = 'immune'
adata.obs['leidencelltype4'] = df_p.leidencelltype4
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_leidencelltype4.png'
sc.pl.umap(adata, color='leidencelltype4',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '),legend_loc='on data')

In [ ]:
df_p.leiden.nunique()

In [ ]:
df_p['annotated'] = df_p.leiden.map(d_named).fillna('_Artifact')
#plot all groups spatially - annotated 
colors = mpl.cm.tab10.colors + mpl.cm.Accent.colors #mpl.cm.tab20b.colors + mpl.cm.tab20c.colors

df_pos = analyze.celltype_to_bool(df_p,'annotated')
ls_clust = df_pos.columns.tolist()#['8','11','12','7','5','10','1','9']#'0','2',
for s_slide in sorted(set(df_p.slide_scene)):
    s_slide = 'U54-TMA-9_sceneG06'
    fig,ax = plt.subplots(figsize=(8,8),dpi=300)
    #plot negative cells
    df_scene = df_data[df_data.index.str.contains(s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.5,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(ls_clust): #enumerate(df_pos.columns):
        #s_color = s_clust
        s_color = str(s_color)
        if len(df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ls_index = df_p[(df_p.slide_scene==s_slide) & (df_pos.loc[:,s_color])].index
            ax.scatter(data=df_data.loc[ls_index],x='DAPI_X',y='DAPI_Y',label=f'{s_color}',s=0.5,color=colors[idxs])
        #break
    ax.set_title(f"{s_slide}", fontsize=16) # \n {d_a[s_slide]}
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=8,framealpha=.5,bbox_to_anchor=(1,1)) 
    fig.savefig(f'{codedir}/{s_date}/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
#save
s_out = f'{s_sample}_LeidenClustering_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv'
df_out = df_p
if  os.path.exists(s_out):
    print(f'saving csv {s_out}')
    df_out.to_csv(s_out)

In [ ]:
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers25_Her2B-K157-6S.csv'
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers24_Her2B-K154-6S.csv'
df_out

## Summarize Tissue Variables <a name="tissue"></a>

per patient means


[contents](#contents)

In [ ]:
os.chdir(datadir)
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers25_Her2B-K157-6S.csv'
#s_out = '20240523_Her2B-K154-157_s_LeidenClustering_neighbors30_resolution0.6_markers24_Her2B-K154-6S.csv'
df_lei['matched']=df_lei.leidencelltype4.copy()
df_lei.loc[df_lei.annotated.str.contains('B cell'),'matched'] = 'B cell'
df_lei.loc[df_lei.annotated.str.contains('T cell'),'matched'] = 'T cell'

In [ ]:
s_sample ='20240528_U54-TMA-Normal-cycIF_s'
#leiden cell types (fraction)
n_neighbors=30
resolution= 0.6

s_type= 'cycIF'#'Her2B-K157-6S'#
s_subtype = s_type
if s_type == 'Her2B-K157-6S':
    n_markers=25
else:
    n_markers=22

df_lei = pd.read_csv(f'{s_sample}_LeidenClustering_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv',index_col=0)
df_lei['slide_scene'] = [item.split('_cell')[0] for item in df_lei.index]
df_lei['leidencelltype2'] = df_lei.leidencelltype3.replace({'tumor':'epithelial','endothelial':'stromal','immune':'stromal'})
df_lei['celltype1'] = 'all'
df_lei['countme'] = True
df_lei['matched']=df_lei.leidencelltype4.copy()
df_lei.loc[df_lei.annotated.str.contains('B cell'),'matched'] = 'B cell'
df_lei.loc[df_lei.annotated.str.contains('T cell'),'matched'] = 'T cell'
df_lei.loc[df_lei.annotated == 'Ductal','matched'] = 'Ductal'
#leiden
#leiden
for s_celltype in ['celltype1']: #'celltype3','celltype','leidencelltype2',
    for s_cell in df_lei.loc[:,s_celltype].unique():
        for s_grouper in ['slide_scene','celltype1']:
            df_cell = df_lei.loc[df_lei.loc[:,s_celltype]==s_cell]
            df_prop = viz.prop_positive(df_cell,s_cell='matched',s_grouper=s_grouper)
            s_out = f'results_{s_sample}_LeidenClustering_{n_neighbors}_{n_markers}_{resolution}_by{s_grouper}_by{s_celltype}_in{s_cell}_{s_type}.csv'
            df_prop.fillna(0).to_csv(s_out)
            print(s_out)

## compare tumor, border and normal

In [ ]:
from statannotations.Annotator import Annotator
import statsmodels
def annotated_stripplot_hue(df,x,y,hue,figsize,hue_order=None,b_correct=True,s=2):
    '''
    with hue, show pvalues, default FDR corrected 
    '''
    order = df.loc[:,x].unique()
    if hue_order ==  None:
        hue_order = df.loc[:,hue].unique()
    plotting = {"data":df,"x":x,"y":y,"order":order,
               "hue":hue,"hue_order":hue_order}
    pairs = [((item,hue_order[0]),(item,hue_order[1])) for item in order]

    fig,ax = plt.subplots(dpi=200,figsize=figsize)
    f,a = plt.subplots()
    sns.stripplot(**plotting,dodge=True,ax=ax,s=s,alpha=0.7)
    sns.boxplot(**plotting,ax=ax,showmeans=True,medianprops={'visible': False},
                                   whiskerprops={'visible': False},meanline=True,showcaps=False,
                           meanprops={'color': 'k', 'ls': '-', 'lw': 2},showfliers=False,showbox=False) 
    annot = Annotator(a,pairs,**plotting,verbose=False)
    h, l = ax.get_legend_handles_labels()
    ax.legend(h[0:len(hue_order)],l[0:len(hue_order)],bbox_to_anchor=(1,1),title=hue)
    annot.configure(test='t-test_ind',text_format="star")
    annot.apply_test()
    a, test_results = annot.apply_test().annotate()
    plt.close(f)
    annot = Annotator(ax,pairs,**plotting,verbose=False)
    d_pval = {}
    for res in test_results:
        d_pval.update({res.data.group1[0]:res.data.pvalue})
    pvalues = [d_pval[item] for item in order]
    reject, corrected, __, __ = statsmodels.stats.multitest.multipletests(pvalues,method='fdr_bh')
    formatted_pvalues = [f'p={pvalue:.2}' for pvalue in list(pvalues)]
    if b_correct:
        formatted_pvalues = [f'FDR={pvalue:.2}' for pvalue in list(corrected)]
    annot.set_custom_annotations(formatted_pvalues)
    annot.annotate()
    return(fig, ax)

def name_celltype(str):
    str = str.replace('Normal breast luminal','Norm. breast').replace('myoep',
        'myoep.').title().replace('Cd','CD').replace('Coli','ColI').replace('Er+','ER+').replace('Fb','Fb.')
    return(str)

In [ ]:
#d_tissue = dict(zip(df_lei.slide_scene, df_lei.Tissue))
d_tissue= {'U54-TMA-9_sceneA01': 'Normal_Placenta',
 'U54-TMA-9_sceneC07': 'Normal_Liver',
 'U54-TMA-9_sceneE12': 'Normal_Liver',
 'U54-TMA-9_sceneG12': 'Normal_Pancreas',
 'U54-TMA-9_sceneH02': 'Normal_Pancreas',
 'U54-TMA-9_sceneB01': 'Normal_Lymphnode',
 'U54-TMA-9_sceneG06': 'Normal_Lymphnode',
 'U54-TMA-9_sceneD03': 'Normal_Tonsil',
 'U54-TMA-9_sceneF08': 'Normal_Tonsil'}

In [ ]:
colors = mpl.cm.tab10.colors + mpl.cm.Accent.colors 
for s_compartment in ['inall']:
    df_file = pd.DataFrame(index=os.listdir())
    df_file = df_file[(df_file.index.str.contains('results_20240528_U54-TMA-Normal')) & (df_file.index.str.contains('byslide_scene'))]
    df_all = pd.DataFrame()
    ls_index = sorted(df_file[df_file.index.str.contains(s_compartment)].index)
    df1=pd.read_csv(ls_index[0],index_col=0)
    df1.index = [item.split('_')[-1] for item in df1.index]
    df2=pd.read_csv(ls_index[1],index_col=0)
    df2.index = [item.split('_')[-1] for item in df2.index]
    s_1 = ls_index[0].split('_')[-1].split('.')[0]
    print(s_1)
    s_2 = ls_index[1].split('_')[-1].split('.')[0]
    fig, ax = plt.subplots(3,3,figsize=(8,10),sharex=True,sharey=True,dpi=300)
    ax=ax.ravel()
    ls_r = []
    for idx, s_index in enumerate(df1.index):
        x = df1.loc[s_index]
        y = df2.loc[s_index]
        r, p = scipy.stats.pearsonr(x.values, y.values)
        ls_r.append(r)
        df = pd.concat([pd.DataFrame(x),pd.DataFrame(y)],axis=1)
        s_tissue = d_tissue['U54-TMA-9_'+df.columns[0]].replace("_"," ")
        s_scene = df.columns[0].replace("scene","")#.replace("00"," ")
        df.columns = [s_1,s_2]
        df.rename({'AF':'_AF'},inplace=True)
        #plot
        title = f'{s_scene.title()}: {s_tissue.title()}\nr={r:.3} p={p:.2}'
        df.T.plot(kind='bar',stacked=True,ax=ax[idx],legend=None,title=title,color=colors)
        if idx < 6:
            ax[idx].tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False) # labels along the bottom edge are off
    ax[idx].legend(bbox_to_anchor=(1,1),ncol=2,title='Cell Type')

    ax[0].set_ylabel(f'Fraction in {s_compartment.split("in")[1]}')
    ax[3].set_ylabel(f'Fraction in {s_compartment.split("in")[1]}')
    ax[6].set_ylabel(f'Fraction in {s_compartment.split("in")[1]}')
    #ax[3].set_xticklabels(['154','157'])

In [ ]:
np.array(ls_r).mean()

In [ ]:
np.array(ls_r).std()

In [ ]:
#all
colors = mpl.cm.tab10.colors + mpl.cm.Accent.colors 
for s_compartment in ['inall']:
    df_file = pd.DataFrame(index=os.listdir())
    df_file = df_file[(df_file.index.str.contains('results_20240528_U54-TMA-Normal')) & (df_file.index.str.contains('_bycelltype1_bycelltype1'))]
    df_all = pd.DataFrame()
    ls_index = sorted(df_file[df_file.index.str.contains(s_compartment)].index)
    df1=pd.read_csv(ls_index[0],index_col=0)
    df1.index = [item.split('_')[-1] for item in df1.index]
    df2=pd.read_csv(ls_index[1],index_col=0)
    df2.index = [item.split('_')[-1] for item in df2.index]
    s_1 = ls_index[0].split('_')[-1].split('.')[0]
    print(s_1)
    s_2 = ls_index[1].split('_')[-1].split('.')[0]
    fig, ax = plt.subplots(figsize=(3,4),sharex=True,sharey=True,dpi=300)
    ls_r = []
    for idx, s_index in enumerate(df1.index):
        x = df1.loc[s_index]
        y = df2.loc[s_index]
        r, p = scipy.stats.pearsonr(x.values, y.values)
        ls_r.append(r)
        df = pd.concat([pd.DataFrame(x),pd.DataFrame(y)],axis=1)
        s_tissue = '' #d_tissue['U54-TMA-9_'+df.columns[0]].replace("_"," ")
        s_scene = df.columns[0].replace("scene","")#.replace("00"," ")
        df.columns = [s_1,s_2]
        df.rename({'AF':'_AF'},inplace=True)
        #plot
        title = f'Adjacent Section Correlation\nr={r:.2} p={p:.2}'
        df.T.plot(kind='bar',stacked=True,ax=ax,legend=None,title=title,color=colors)
        ax.set_title(title,fontsize=16)
    ax.legend(bbox_to_anchor=(1,1),ncol=2,title='Cell Type')